In [1]:
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

train = pd.read_csv("./data/train_feat.csv").drop(['ID'], axis=1)
test = pd.read_csv("./data/test_feat.csv")

train = train.fillna(train.mean())
test = test.fillna(test.mean())

X = train.drop(['Type'], axis=1)
y = train['Type']

train.isnull().sum().sum()

To request implementation, send an email to feature_requests@modin.org.


0

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

# K-Fold Cross Validation Code is adapted from here: https://towardsdatascience.com/why-and-how-to-cross-validate-a-model-d6424b45261f
# Used to scout for any overfitting problems before moving onto using GridSearch to find optimal Hyperparameters
"""
kf = KFold(n_splits = 5, shuffle = True)
randfor = RandomForestClassifier(n_estimators=128, bootstrap=False, max_depth=None, min_samples_split=2)

scores=[]
for i in range(5):
    result = next(kf.split(train), None)
    X_train = X.iloc[result[0]]
    X_test = X.iloc[result[1]]
    y_train = y.iloc[result[0]]
    y_test = y.iloc[result[1]]
    model = randfor.fit(X_train,y_train)
    predictions = randfor.predict(X_test)
    scores.append(model.score(X_test,y_test))
    
print('Scores from each iteration: ', scores)
print('Average K-Fold Score: ', np.mean(scores))
"""

"\nkf = KFold(n_splits = 5, shuffle = True)\nrandfor = RandomForestClassifier(n_estimators=128, bootstrap=False, max_depth=None, min_samples_split=2)\n\nscores=[]\nfor i in range(5):\n    result = next(kf.split(train), None)\n    X_train = X.iloc[result[0]]\n    X_test = X.iloc[result[1]]\n    y_train = y.iloc[result[0]]\n    y_test = y.iloc[result[1]]\n    model = randfor.fit(X_train,y_train)\n    predictions = randfor.predict(X_test)\n    scores.append(model.score(X_test,y_test))\n    \nprint('Scores from each iteration: ', scores)\nprint('Average K-Fold Score: ', np.mean(scores))\n"

In [3]:
"""
from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)



pca = PCA(n_components=0.95) # reduce dimensions to explain 95% of variance as recommended here: https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/
pca.fit(X_train)

X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

print(X_train.shape) 
print(X_test.shape)
"""

'\nfrom sklearn.decomposition import PCA\n\nfrom sklearn.preprocessing import MinMaxScaler\n\nscaler = MinMaxScaler()\nX = scaler.fit_transform(X)\n\n\n\npca = PCA(n_components=0.95) # reduce dimensions to explain 95% of variance as recommended here: https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/\npca.fit(X_train)\n\nX_train = pca.transform(X_train)\nX_test = pca.transform(X_test)\n\nprint(X_train.shape) \nprint(X_test.shape)\n'

In [4]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

# Randomised Grid Search for Random Forest adapted from here: 
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]


random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 2, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)


Fitting 2 folds for each of 10 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:  3.5min remaining:  2.1min


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
sample = pd.read_csv('./data/sampleSubmission.csv')
print(sample.head())
#Make predications on test data
model = rf_random

y_pred = model.predict(test)
results = pd.DataFrame()
results["ID"]=sample["ID"]
results["Predicted"]= y_pred

#Export result
results.to_csv("submissionRandomForest.csv", index=False)

In [ ]:
print(results.head())